In [1]:
import igraph as ig
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
from shapely.geometry import box, Point, LineString, MultiLineString
import os
import osmnx as ox

In [2]:
rail_file_path="C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\euro-global-map-shp\\euro-global-map-shp\\euro-global-map-shp\\RoadL.shp"
rail_stops_file_path="C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\euro-global-map-shp\\euro-global-map-shp\\euro-global-map-shp\\BuiltupP.shp"

gdf_cut = ox.geocode_to_gdf("Cantabria, Spain")
gdf_cut = gdf_cut.to_crs(epsg=4258)

In [3]:
gdf_rail = gpd.read_file(rail_file_path)
gdf_rail = gpd.clip(gdf_rail, gdf_cut)

gdf_rail_stops=gpd.read_file(rail_stops_file_path)
gdf_rail_stops=gpd.clip(gdf_rail_stops, gdf_cut)

In [4]:
'Unir lines'
# 1. Asegurarse de que todas las geometrías son LineString
def explode_multilines(gdf):
    out = []
    for idx, geom in gdf.geometry.items():
        if isinstance(geom, LineString):
            out.append(geom)
        elif isinstance(geom, MultiLineString):
            out.extend(list(geom.geoms))
    return gpd.GeoDataFrame(geometry=out, crs=gdf.crs)

gdf_lines = explode_multilines(gdf_rail)

# 2. Construir grafo topológico de los tramos
G = nx.Graph()
for i, line in enumerate(gdf_lines.geometry):
    coords = list(line.coords)
    for j in range(len(coords) - 1):
        p1, p2 = coords[j], coords[j+1]
        G.add_edge(p1, p2, line_id=i)

# 3. Encontrar cadenas lineales (secuencias con nodos intermedios de grado=2)
def find_chains(G):
    visited_edges = set()
    chains = []

    for u, v in G.edges():
        if (u, v) in visited_edges or (v, u) in visited_edges:
            continue

        chain = [u, v]
        visited_edges.add((u, v))
        visited_edges.add((v, u))

        # expandir hacia u
        current = u
        prev = v
        while G.degree[current] == 2:
            neighbors = [n for n in G.neighbors(current) if n != prev]
            if not neighbors:
                break
            nxt = neighbors[0]
            chain.insert(0, nxt)
            visited_edges.add((current, nxt))
            visited_edges.add((nxt, current))
            prev, current = current, nxt

        # expandir hacia v
        current = v
        prev = u
        while G.degree[current] == 2:
            neighbors = [n for n in G.neighbors(current) if n != prev]
            if not neighbors:
                break
            nxt = neighbors[0]
            chain.append(nxt)
            visited_edges.add((current, nxt))
            visited_edges.add((nxt, current))
            prev, current = current, nxt

        chains.append(chain)

    return chains

chains = find_chains(G)

# 4. Crear nuevas LineString fusionadas
merged_lines = [LineString(chain) for chain in chains]

gdf_rail = gpd.GeoDataFrame(geometry=merged_lines, crs=gdf_lines.crs)

#gdf_rail.to_file("vias_fusionadas.shp")

In [ ]:
'Plotear con lines unidas'
gdf_rail = gdf_rail.reset_index(drop=True)   # por si acaso limpiar el índice
gdf_rail["id"] = gdf_rail.index              # usar el índice como id
ax = gdf_rail.plot(
    column="id",
    cmap="tab20",
    linewidth=2,
    legend=True,
    figsize=(10, 8)
)

plt.show()

In [ ]:
'Plotar gdfs'
fig, ax = plt.subplots(figsize=(10, 8))
gdf_cut.plot(ax=ax,color="lightgrey", edgecolor="black", figsize=(8, 8))
gdf_rail.plot(ax=ax, color="green", edgecolor="black",linewidth=0.5)
gdf_rail_stops.plot(ax=ax, color="red",markersize=0.5)

plt.savefig("mapa.png", dpi=300, bbox_inches="tight")
plt.close()
os.startfile("mapa.png")

In [5]:
'GDF to NX_GRAPH'

# Grafo multiarista
G = nx.MultiGraph()

# Añadir nodos de estaciones
for idx, est in gdf_rail_stops.iterrows():
    G.add_node(idx, **est.to_dict(), type="station")

# Diccionario para localizar nodos por coordenadas (tuplas)
coord_to_node = {tuple(est.geometry.coords[0]): idx for idx, est in gdf_rail_stops.iterrows()}

cross_id = 0
buffer_distance = 0.01  # tolerancia alrededor de la vía

# Diccionario para registrar la mejor vía asignada a cada estación
station_to_best_via = {}

# --- Primera pasada: calcular mejor vía para cada estación ---
for i, via in gdf_rail.iterrows():
    via_buffer = via.geometry.buffer(buffer_distance)

    # Estaciones dentro del buffer
    estaciones_cercanas = gdf_rail_stops[gdf_rail_stops.geometry.intersects(via_buffer)].copy()

    if not estaciones_cercanas.empty:
        estaciones_cercanas['distance_to_via'] = estaciones_cercanas.geometry.apply(lambda p: via.geometry.distance(p))

        for idx, est in estaciones_cercanas.iterrows():
            dist = est['distance_to_via']
            if (idx not in station_to_best_via) or (dist < station_to_best_via[idx]["dist"]):
                station_to_best_via[idx] = {"via": i, "dist": dist}

    # --- Crear nodos inicial/final de la vía ---
    line_start = tuple(via.geometry.coords[0])
    if line_start in coord_to_node:
        start_node = coord_to_node[line_start]
    else:
        start_node = f"start_{cross_id}"
        G.add_node(start_node, geometry=Point(line_start), type="start")
        coord_to_node[line_start] = start_node
        cross_id += 1

    line_end = tuple(via.geometry.coords[-1])
    if line_end in coord_to_node:
        end_node = coord_to_node[line_end]
    else:
        end_node = f"end_{cross_id}"
        G.add_node(end_node, geometry=Point(line_end), type="end")
        coord_to_node[line_end] = end_node
        cross_id += 1

    # Guardamos extremos para referencia
    G.nodes[start_node][f"via_{i}_endpoint"] = "start"
    G.nodes[end_node][f"via_{i}_endpoint"] = "end"

# --- Segunda pasada: asignar vías definitivas y crear aristas ---
for station, best in station_to_best_via.items():
    G.nodes[station]['assigned_via'] = best["via"]

for i, via in gdf_rail.iterrows():
    # Estaciones que quedaron asignadas a esta vía
    estaciones_via = [s for s, best in station_to_best_via.items() if best["via"] == i]

    if len(estaciones_via) > 0:
        subset = gdf_rail_stops.loc[estaciones_via].copy()
        subset["pos"] = subset.geometry.apply(lambda p: via.geometry.project(p))
        estaciones_ordenadas = subset.sort_values("pos").index.tolist()

        # Conectar estaciones consecutivas
        for j in range(len(estaciones_ordenadas) - 1):
            n1 = estaciones_ordenadas[j]
            n2 = estaciones_ordenadas[j + 1]
            G.add_edge(n1, n2, via_id=i)

        # Conectar extremos con estaciones
        line_start = tuple(via.geometry.coords[0])
        line_end   = tuple(via.geometry.coords[-1])
        G.add_edge(coord_to_node[line_start], estaciones_ordenadas[0], via_id=i)
        G.add_edge(estaciones_ordenadas[-1], coord_to_node[line_end], via_id=i)
    else:
        # Sin estaciones asignadas: conectar extremos
        line_start = tuple(via.geometry.coords[0])
        line_end   = tuple(via.geometry.coords[-1])
        G.add_edge(coord_to_node[line_start], coord_to_node[line_end], via_id=i)

# --- Colapsar nodos de intersección ---
nodes_to_collapse = [n for n, d in G.degree() if d > 2]

for n in nodes_to_collapse:
    neighbors = list(G.neighbors(n))

    # Conectar todos los vecinos entre sí con MultiGraph (cada conexión será una arista adicional)
    for i in range(len(neighbors)):
        for j in range(i + 1, len(neighbors)):
            G.add_edge(neighbors[i], neighbors[j], via_id=f"collapsed_from_{n}")

    # Eliminar nodo central
    G.remove_node(n)

# --- Eliminar componentes conexas de solo 2 o 3 nodos ---
for comp in list(nx.connected_components(G)):
    if len(comp) == 2 or len(comp)==3:  # es solo una arista
        G.remove_nodes_from(comp)


In [ ]:
'Plotear NX_GRAPH'
fig, ax = plt.subplots(figsize=(10, 8))
gdf_cut.plot(ax=ax, color="lightgrey", edgecolor="black")

pos = {n: (G.nodes[n]['geometry'].x, G.nodes[n]['geometry'].y) for n in G.nodes()}
G_simple = nx.Graph()
for u, v in G.edges():
    G_simple.add_edge(u, v)  # solo 1 arista por par de nodos

nx.draw(G_simple, pos, node_size=0.1, width=0.1, node_color="red",
        edge_color="green", with_labels=False, ax=ax)

# Guardar imagen
plt.savefig("grafo_nx.png", dpi=1200, bbox_inches="tight")
plt.close()
os.startfile("grafo_nx.png")

In [ ]:
'''Plotear nx abstracto'''
fig, ax = plt.subplots(figsize=(10, 8))

G_simple = nx.Graph()
for u, v in G.edges():
    G_simple.add_edge(u, v)  # solo 1 arista por par de nodos

pos = nx.spring_layout(G, seed=42)   # layout abstracto
nx.draw(
    G, pos,
    node_size=0.5,
    width=0.5,
    node_color='red',
    edge_color='blue',
    with_labels=False,
    ax=ax
)
# Guardar imagen
plt.savefig("grafo_nx_abs.png", dpi=300, bbox_inches="tight")
plt.close()
os.startfile("grafo_nx_abs.png")

In [6]:
'NX_GRAPH to IG'
 # 1️⃣ Lista de nodos y mapeo a índices
nodes = list(G.nodes())
node_to_idx = {n: i for i, n in enumerate(nodes)}

# 2️⃣ Extraer todas las aristas (incluidas paralelas) y atributos
edges = []
edges_attrs = []
for u, v, key, attr in G.edges(keys=True, data=True):
    edges.append((node_to_idx[u], node_to_idx[v]))
    edges_attrs.append(attr.copy())  # copiar el diccionario de atributos

# 3️⃣ Crear grafo igraph (duplicados de edges se mantienen automáticamente)
g_ig = ig.Graph(edges=edges, directed=False)

# 4️⃣ Añadir atributos de nodos
for n, idx in node_to_idx.items():
    g_ig.vs[idx]["name"] = n  # opcional: nombre original del nodo
    for k, v in G.nodes[n].items():
        g_ig.vs[idx][k] = v

# 5️⃣ Añadir atributos de aristas
for e_idx, attr in enumerate(edges_attrs):
    for k, v in attr.items():
        g_ig.es[e_idx][k] = v

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
gdf_cut.plot(ax=ax, color="lightgrey", edgecolor="black")
coords = g_ig.vs["coords"]
ig.plot(g_ig,layout=coords,target=ax,vertex_size=1,edge_width=0.5,vertex_color="red",edge_color="black")
#ig.plot(g_ig,layout=coords,target=ax,vertex_size=2,edge_width=0.5,vertex_color="red",
#        edge_color="gray",vertex_label=[str(v.index) for v in g_ig.vs],vertex_label_size=2,vertex_label_color="black")
#ig.plot(g_ig,layout=coords,target=ax,vertex_size=2,edge_width=g_ig.es['width'],vertex_color="red",edge_color=g_ig.es['colour'],vertex_label=labels,vertex_label_size=10,vertex_label_color="black")
xmin, ymin, xmax, ymax = bbox.total_bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
plt.savefig("grafo_ig.png", dpi=300, bbox_inches="tight")
os.startfile("grafo_ig.png")

In [ ]:
"""
Dibuja un grafo no dirigido mostrando flechas simuladas desde cada nodo
para indicar el flujo que sale por cada arista.
Cada arista tiene dos atributos: flujo desde cada nodo.
"""

# Extraer coordenadas
x_coords = [v['geometry'].x for v in g_ig.vs]
y_coords = [v['geometry'].y for v in g_ig.vs]
coords_layout = list(zip(x_coords, y_coords))

#layout = g_ig.layout("kk")
#coords_layout = [(x, y) for x, y in layout.coords]
x_coords = [c[0] for c in coords_layout]
y_coords = [c[1] for c in coords_layout]

# Creamos listas para flechas y etiquetas
arrows = []
edge_labels = []
edge_widths = []

for e in g_ig.es:
    u_idx, v_idx = e.source, e.target
    u_name = str(g_ig.vs[u_idx]["name"])
    v_name = str(g_ig.vs[v_idx]["name"])

    # Flujo desde cada nodo
    flow_u = e[u_name]
    flow_v = e[v_name]

    # Simulamos flecha desde u a v
    arrows.append(((coords_layout[u_idx], coords_layout[v_idx]), flow_u))
    # Simulamos flecha desde v a u
    arrows.append(((coords_layout[v_idx], coords_layout[u_idx]), flow_v))

# Plot con matplotlib
fig, ax = plt.subplots(figsize=(12,10))
ax.set_aspect('equal')
ax.set_title("Flujos de nodos")

# Dibujar nodos
ax.scatter(x_coords, y_coords, s=5, c='red', zorder=3)
for i, v in enumerate(g_ig.vs):
    ax.text(
        x_coords[i],
        y_coords[i],
         f"{v['name']}\n({v['users']})",  # Texto: nombre (usuarios)
        fontsize=1,
        color="black",        # Contrasta mejor con el nodo rojo
        ha="center",          # Centrado horizontal
        va="center",          # Centrado vertical
        zorder=4,
    )


# Dibujar flechas y etiquetas
for (start, end), flow in arrows:
    # Dibujar la flecha
    ax.annotate(
        "", xy=end, xytext=start,
        arrowprops=dict(arrowstyle="->", color='blue', lw=0.8),
        zorder=2
    )

    # Vector de la arista
    dx = end[0] - start[0]
    dy = end[1] - start[1]
    length = (dx**2 + dy**2)**0.5

    # Posición del texto: cerca del inicio, a un 15% de la arista
    t = 0.15
    x_text = start[0] + t * dx
    y_text = start[1] + t * dy

    # Desplazamiento perpendicular opcional para evitar solapamientos
    perp_offset = 0.02 * length
    x_text += -perp_offset * dy / length
    y_text += perp_offset * dx / length

    # Dibujar la etiqueta
    ax.text(x_text, y_text, f"{flow:.2f}",
            fontsize=1, color='black', zorder=4,
            ha='center', va='center')

gdf_cut.plot(ax=ax, facecolor="none", edgecolor="black")
xmin, ymin, xmax, ymax = gdf_cut.total_bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
plt.savefig("grafo_ig_flujo.png", dpi=300, bbox_inches="tight")
plt.close()
os.startfile("grafo_ig_flujo.png")